In [1]:
# import os
# os.remove("./seed/normalized")
# normalized_rankings = open("./seed/normalized", 'w+')
# normalized_rankings.write("title,user,rating\n")
#

Netflix normalization

In [2]:
# import pandas as pd
#
# print("normalizing data...")
#
# titles_dict = {'id': [], 'year': [], 'title': []}
# with open("./seed/netflix-prize/movie_titles.txt", 'r') as titles_file:
#     for idx, line in enumerate(titles_file.readlines()):
#         split = line.split(',', 2)
#
#         if split[1] == 'NULL':
#             continue
#
#         titles_dict['id'].append(int(split[0]))
#         titles_dict['year'].append(int(split[1]))
#         titles_dict['title'].append(split[2][:-1])
#
# titles_df = pd.DataFrame(titles_dict)
# titles_df = titles_df.set_index('id')
#
# ts_count = len([name for name in os.listdir('./seed/netflix-prize/training_set') if os.path.isfile(name)])
# for index, row in titles_df.iterrows():
#
#     if (index / ts_count) % 1000 == 0:
#         print(f"{index}")
#
#     with open(f"./seed/netflix-prize/training_set/mv_{index:07}.txt", 'r') as train_file:
#         train_file.readline()
#         for line in train_file:
#             split = line.split(',', 2)
#             user = split[0]
#             rating = split[1]
#             date = split[2]
#
#             normalized_rankings.write(f"{row['title']},n-{user},{rating}\n")
#
#
#
#

Trying TensorFlow

In [7]:
from datasets.couchtime.couchtime import parse_current_movies_data
from typing import Dict, Text

import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

#initialize datasets
# import datasets.couchtime
# import datasets.mlens
# import datasets.shows

# Ratings data.
zam = tfds.load('couchtime', split="train", data_dir='datasets/')
ratings = tfds.load('mlens', split="train", data_dir='datasets/').concatenate(zam)
# ratings = tfds.load('movielens/latest-small-ratings', split="train")

# Features of all the available movies.
movies = tfds.load('shows', split="train", data_dir='datasets/')
# movies = tfds.load('movielens/latest-small-movies', split="train")

# Select the basic features.
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"]
})
movies = movies.map(lambda x: x["movie_title"])

user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))

movie_titles_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
movie_titles_vocabulary.adapt(movies)

class MovieLensModel(tfrs.Model):
    # We derive from a custom base class to help reduce boilerplate. Under the hood,
    # these are still plain Keras Models.

    def __init__(
            self,
            user_model: tf.keras.Model,
            movie_model: tf.keras.Model,
            task: tfrs.tasks.Retrieval):
        super().__init__()

        # Set up user and movie representations.
        self.user_model = user_model
        self.movie_model = movie_model

        # Set up a retrieval task.
        self.task = task

    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
        # Define how the loss is computed.

        user_embeddings = self.user_model(features["user_id"])
        movie_embeddings = self.movie_model(features["movie_title"])

        return self.task(user_embeddings, movie_embeddings)

# Define user and movie models.
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocabulary_size(), 64)
])
movie_model = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocabulary_size(), 64)
])

# Define your objectives.
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    movies.batch(128).map(movie_model)
))

# Create a retrieval model.
model = MovieLensModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# Train for 3 epochs.
model.fit(ratings.batch(4096), epochs=3)

# Use brute-force search to set up retrieval using the trained representations.
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
    movies.batch(100).map(lambda title: (title, model.movie_model(title))))

_, titles = index(np.array(["8888444"]))
print(f"Top 3 recommendations for zack: {titles[0, :20]}")

Epoch 1/3
25/25 [==============================] - 35s 1s/step - factorized_top_k/top_1_categorical_accuracy: 9.9168e-06 - factorized_top_k/top_5_categorical_accuracy: 5.5534e-04 - factorized_top_k/top_10_categorical_accuracy: 0.0019 - factorized_top_k/top_50_categorical_accuracy: 0.0182 - factorized_top_k/top_100_categorical_accuracy: 0.0401 - loss: 33384.7090 - regularization_loss: 0.0000e+00 - total_loss: 33384.7090         